[pickle](https://docs.python.org/2/library/pickle.html) mechanism.

## The Basics of Pickling

* pickle? 오이 같은 것을 절여서 보존하다.
* pickle module? 이 모듈을 이용하여 파이썬 객체를 byte stream으로 변환하여 데이터를 byte단위로 영구적으로 저장하거나, 데이터베이스에 저장하거나 네트워크로 전송할 수 있다.

기본 파이썬 객체 뿐만 아니라 객체들의 관계도 저장한다.

일반 문자열 같은 경우 f.write() 함수를 사용할 수 있지만, 튜플, 리스트같은 다른 객체들은 이러한 방법으로 파일에 저장할 수 없다.

### 1. pickle.dump 

* 객체를 byte stream으로 변환하여 데이터를 byte단위로 **저장**하는 것을 **pickling**(or serialization) 이라고 함.

In [14]:
import cPickle

- pickle과 cPickle 모듈이 있다. 비슷한 기능이지만 cPickle은 c로 coding되어 있어 1000배 정도 더 빠르고 두 모듈이 생성한 data stream은 호환된다.

- file을 binary mode('b')로 open하는 것은, 특히 window와 unix 사이에서 호환되기 위해 필요하다.

- cPickle.HIGHEST_PROTOCOL을 사용하면 더 효율적으로 object를 저장할 수 있다.

In [15]:
my_obj = "test"
f = file('obj.save','wb')
cPickle.dump(my_obj, f, protocol=cPickle.HIGHEST_PROTOCOL) # pickling
f.close()
#with open('obj.save', 'wb') as f:
#    cPickle.dump(my_obj, f)

### 2. pickle.load
다시 **loading**하여 객체를 재구성하는 것을 **unpickling**(or de-serialization)으로 부름

In [16]:
f = file('obj.save','rb')
loaded_obj = cPickle.load(f)  # un-pickling
f.close()

#with open('obj.save', 'rb') as f:
#    loaded_obj = cPickle.dump(f)

print loaded_obj

test


여러개 object들을 하나의 파일에 저장하고 loading할 수 있다.

In [17]:
obj1=100                   # numeric
obj2={"test1":1,"test2":2} # dictionary
obj3=("test1","test2")     # tuple

f = file('objects.save','wb')
for obj in [obj1, obj2, obj3]:
    cPickle.dump(obj, f, protocol=cPickle.HIGHEST_PROTOCOL) # pickling
f.close()

In [18]:
f = file('objects.save','rb')
loaded_objects=[]
for i in range(3):
    loaded_objects.append(cPickle.load(f))  # un-pickling

print loaded_objects
f.close()

[100, {'test1': 1, 'test2': 2}, ('test1', 'test2')]


-피클링(Pickling)할 수 있는 파이썬 객체들

	*None
	*integer, long integers, floating point numbers
	*normal and Unicode strings
	*tuples, lists and dictionaries containing only picklable object
	*Functions defined at the top level of a module (by name reference, not stoage of the implementation)
	*built-in fuctions
	*classes that are defined at the top level in a module
	*instances of such classes whose __dict__ or __setstate__() is picklable


## Short-Term Serialization

pickling은 그냥 단순 텍스트 파일을 사용하는 것보다 더 안전할 수 있지만, 파이썬 버전마다 호환이 되지 않을 때도있고, **임시 저장**을 위한 용도로 강력한 도구이다.

(하나의 program을 실행하는 동안 saving & reloading하는 용도)

아래와 같이 class를 pickling할 때에는 class object의 \_\_getstate\_\_ 와 \_\_setstate\_\_ method를 정의하여 유용하게 이용할 수 있다.

 - class object를 picking할 때 \_\_getstate\_\_(self) 함수가 호출
 - un-pickling할 때 \_\_setstate\_\_(self, state) 함수가 호출

In [129]:
import cPickle

class Foo(object):
    # class 생성시 호출되는 함수
    def __init__(self, val1=1, val2=2):
        self.w = val1
        self.b = val2
  # pickling 용도로 호출되는 함수
    def __getstate__(self):
        print "getstate implemented.."
        self.old = (self.w , self.b)
        return self.__dict__
    def __setstate__(self, state):
        print "setstate implemented"
        print "get state: ", state
        self.__dict__ = state
        self.b = 0
        self.w = 0

foo = Foo()
print foo.__dict__

{'b': 2, 'w': 1}


In [146]:
with open('obj.save','wb') as f:
    cPickle.dump(foo, f) # pickling

print foo.__dict__

getstate implemented..
{'b': 2, 'old': (1, 2), 'w': 1}


* class를 pickling하게되면 ? ... \_\_getstate\_\_(self)로 정의된 내용이 수행되고 pickling될 때 특정 값(state)를 return함 

* 그 class를 다시 un-pickling하게되면 ? ... \_\_setstate\_\_(self, state)로 정의된 내용이 수행되고, getstate가 return한 값을 받아서 처리한다.

In [131]:
with open('obj.save','rb') as f:
    loaded_foo = cPickle.load(f)  # un-pickling

print loaded_foo.__dict__

setstate implemented
get state:  {'b': 2, 'old': (1, 2), 'w': 1}
{'b': 0, 'old': (1, 2), 'w': 0}


In [143]:
class myClass(object):
    def __init__(self, val1=[1,2,3], val2=[4,5,6]):
        self.training_set = val1
        self.test_set = val2
    def __getstate__(self):
        print "getstate implement.."
        state = self.__dict__
        del state['training_set']
        return state
    def __setstate__(self, state):
        print "setstate implemented.."
        self.__dict__ = state
        self.training_set = [0,0,0]

bef = myClass(obj)
with open('obj.save', 'wb') as f:
    cPickle.dump(bef, f)
print bef.__dict__
print "==="
with open('obj.save', 'rb') as f:
    aft =cPickle.load(f)
print aft.__dict__

getstate implement..
{'test_set': [4, 5, 6]}
===
setstate implemented..
{'test_set': [4, 5, 6], 'training_set': [0, 0, 0]}


## Long-Term Serialization

장기적으로 보면 class에 정의 된 member(변수)들이나 method(함수)들은 remove, rename, 등 변경될 가능성이 있으므로

class 중에서도 변하지 않는 (immutable) 부분만 저장하는 경우가 많다.

아래와 같이 메쏘드를 정의하면, W와 b만 save & reload 할 수 있다.

In [170]:
class myClass(object):
    def __init__(self, w=1, b=0):
        self.w = w
        self.b = b
    def __getstate__(self):
        return (self.w, self.b)
    def __setstate__(self, state):
        w, b = state
        self.w = w
        self.b = b
        
bef = myClass()
with open('obj1.save', 'wb') as f:
    cPickle.dump(bef, f)             # pickling
print bef.__dict__
print "==="
with open('obj1.save', 'rb') as f:
    aft =cPickle.load(f)             # un-pickling
print aft.__dict__

{'b': 0, 'w': 1}
===
{'b': 0, 'w': 1}


예전에 저장해두었던 pickled class인 myClass()의 instance 에서 변수명을 w, b로 사용하였었는데,

어느 순간 w-> weights, b-> bias로 rename해서 사용해야할 경우

예전에 pickled 해 놓았던 file을 그대로 유효하게 읽어온 뒤에 아래와 같이 instance의 변수명을 바꿔서 읽어들일 수 있다.

In [173]:
class myClass(object):
    def __init__(self, w=1, b=0):
        self.w = w
        self.b = b
    def __getstate__(self):
        return (self.w, self.b)
    def __setstate__(self, state):
        w, b = state
        self.weigthts = w
        self.bias = b
        
with open('obj1.save', 'rb') as f:
    updated =cPickle.load(f)             # un-pickling
        
print updated.__dict__

{'bias': 0, 'weigthts': 1}


## Robust Serialization

* theano.misc.pkl_utils.dump()
* theano.misc.pkl_utils.load()

위 함수들은 .npy file들(ndarray, CudaNdarray object)과 pickled file들을 하나의 **ZIP file**에 저장한다. 

이 방법의 주요 장점은, 다른 사람들과 **데이터를 공유**하기 유용하고 장기간 보관하기 유용하다는 것이다. 이 파일들의 내용(shared 변수 등)을 보기 위해서 Theano를 설치할 필요 없이 **numpy로 loading 가능**하기 때문.

In [37]:
# [참고] .npy file
import numpy as np
arr = np.array(2)
print "type:", type(arr)
print "input:",arr
np.save('array',arr)
out =np.load('array.npy')
print "output:",out

type: <type 'numpy.ndarray'>
input: 2
output: 2


In [38]:
import numpy
import theano
from pkl_utils_old import dump, load
foo_1 = theano.shared(0, name='foo1')
foo_2 = theano.shared(1, name='foo2')

with open('model.zip', 'w') as f:
    dump((foo_1, foo_2, numpy.array(3)), f)

with open('model.zip') as f:
    foo_11, foo_22, arr = load(f)
print foo_22.get_value()

1


In [39]:
print numpy.load('model.zip').keys()
print numpy.load('model.zip')['foo2']

['foo1', 'foo2', 'array_0', 'pkl']
1
